# NRPyEllipCriticlapse
A frankenstein of NRPyEllipticTutorial & Javascript Interface Code for Scalar Collapse.

In [2]:
# nrpytutorial should be in a subdirectory, so, do the following.
import os,sys
import shutil
#nrpy_dir_path = os.path.join("nrpytutorial/BSSN")
#if nrpy_dir_path not in sys.path:
#    sys.path.append(nrpy_dir_path)

nrpyellipt_dir_path = os.path.join("Elliptic/NRPyElliptic")
if nrpyellipt_dir_path not in sys.path:
    sys.path.append(nrpyellipt_dir_path)

try: 
    shutil.rmtree("BSSN", ignore_errors=True)
    shutil.rmtree("ScalarField",ignore_errors=True)
except:
    pass

In [ ]:
# Step P1: Import needed NRPy+ core modules:
#from outputC import lhrh,outputC,outCfunction  # NRPy+: Core C code output module
#import NRPy_param_funcs as par   # NRPy+: Parameter interface
#import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
#import finite_difference as fin  # NRPy+: Finite difference C code generation module
#import grid as gri               # NRPy+: Functions having to do with numerical grids
#import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
#import reference_metric as rfm   # NRPy+: Reference metric support
#import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface


# Step P1: Import needed NRPy+ core modules:
import sympy as sp                    # SymPy: The Python computer algebra package upon which NRPy+ depends 
from outputC import lhrh, add_to_Cfunction_dict  # NRPy+: Core C code output module
import finite_difference as fin       # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par        # NRPy+: Parameter interface
import indexedexp as ixp              # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import grid as gri                    # NRPy+: Functions having to do with numerical grids
from outputC import outputC,lhrh,outCfunction # NRPy+: Core C code output module
import loop as lp                     # NRPy+: C code loop interface
import reference_metric as rfm        # NRPy+: Reference metric support
import cmdline_helper as cmd          # NRPy+: Multi-platform Python command-line interface 

# Step P2: Create C code output directory:
Ccodesdir = os.path.join("SommerBSSN_Scalar_Collapse/")
# First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
shutil.rmtree(Ccodesdir, ignore_errors=True)
# Then create a fresh directory
cmd.mkdir(Ccodesdir)

# Step P3: Create executable output directory:
outdir = os.path.join(Ccodesdir,"output/")
cmd.mkdir(outdir)

# Step 1: Set the spatial dimension parameter
#         to three (BSSN is a 3+1 decomposition
#         of Einstein's equations), and then read
#         the parameter as DIM.
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 1.a: Enable SIMD-optimized code?
#           I.e., generate BSSN and Ricci C code kernels using SIMD-vectorized
#           compiler intrinsics, which *greatly improve the code's performance*,
#           though at the expense of making the C-code kernels less
#           human-readable.
#           * Important note in case you wish to modify the BSSN/Ricci kernels
#             here by adding expressions containing transcendental functions
#             (e.g., certain scalar fields):
#           Note that SIMD-based transcendental function intrinsics are not
#           supported by the default installation of gcc or clang (you will
#           need to use e.g., the SLEEF library from sleef.org, for this
#           purpose). The Intel compiler suite does support these intrinsics
#           however without the need for external libraries.
#enable_SIMD = True

# Step P4: Enable/disable SIMD. If enabled, code should run ~2x faster on most CPUs.
enable_SIMD = False

# Step P5: Enable reference metric precomputation.
enable_rfm_precompute = False

if enable_rfm_precompute:
    par.set_parval_from_str("reference_metric::rfm_precompute_to_Cfunctions_and_NRPy_basic_defines", "True")
else:
    par.set_parval_from_str("reference_metric::rfm_precompute_to_Cfunctions_and_NRPy_basic_defines", "False")
    
if enable_SIMD and not enable_rfm_precompute:
    print("ERROR: SIMD does not currently handle transcendental functions,\n")
    print("       like those found in rfmstruct (rfm_precompute).\n")
    print("       Therefore, enable_SIMD==True and enable_rfm_precompute==False\n")
    print("       is not supported.\n")
    sys.exit(1)

# Step 2: Set some core parameters, including CoordSystem MoL timestepping algorithm,
#                                 FD order, floating point precision, and CFL factor:
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical,
#              SymTP, SinhSymTP
#CoordSystem     = "Spherical"

# Step P6: Enable "FD functions". In other words, all finite-difference stencils
#         will be output as inlined static functions. This is essential for
#         compiling highly complex FD kernels with using certain versions of GCC;
#         GCC 10-ish will choke on BSSN FD kernels at high FD order, sometimes
#         taking *hours* to compile. Unaffected GCC versions compile these kernels
#         in seconds. FD functions do not slow the code performance, but do add
#         another header file to the C source tree.
# With gcc 7.5.0, enable_FD_functions=True decreases performance by 10%
enable_FD_functions = False

# domain_size sets the default value for:
#   * Spherical's params.RMAX
#   * SinhSpherical*'s params.AMAX
#   * Cartesians*'s -params.{x,y,z}min & .{x,y,z}max
#   * Cylindrical's -params.ZMIN & .{Z,RHO}MAX
#   * SinhCylindrical's params.AMPL{RHO,Z}
#   * *SymTP's params.AMAX
#domain_size = 10 # Length scale of computational domain
#FD_order    = 8   # Finite difference order: even numbers only, starting with 2. 12 is generally unstable

# Step 1: Set some core parameters, including CoordSystem, boundary condition,
#                                             MoL, timestepping algorithm, FD order,
#                                             floating point precision, and CFL factor:

# Step 1.a: Set the coordinate system for the numerical grid
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical,
#              SymTP, SinhSymTP
CoordSystem     = "Spherical"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

# Step 1.b: Set boundary conditions
# Currently Sommerfeld is the only supported BC driver
OuterBoundaryCondition = "Sommerfeld"

# Step 1.c: Set defaults for Coordinate system parameters.
#           These are perhaps the most commonly adjusted parameters,
#           so we enable modifications at this high level.

# domain_size sets the default value for:
#   * Spherical's params.RMAX
#   * SinhSpherical*'s params.AMAX
#   * Cartesians*'s -params.{x,y,z}min & .{x,y,z}max
#   * Cylindrical's -params.ZMIN & .{Z,RHO}MAX
#   * SinhCylindrical's params.AMPL{RHO,Z}
#   * *SymTP's params.AMAX
domain_size     = 10 # Needed for all coordinate systems.

# sinh_width sets the default value for:
#   * SinhSpherical's params.SINHW
#   * SinhCylindrical's params.SINHW{RHO,Z}
#   * SinhSymTP's params.SINHWAA
sinh_width      = 0.2 # If Sinh* coordinates chosen

# sinhv2_const_dr sets the default value for:
#   * SinhSphericalv2's params.const_dr
#   * SinhCylindricalv2's params.const_d{rho,z}
sinhv2_const_dr = 0.05# If Sinh*v2 coordinates chosen

# SymTP_bScale sets the default value for:
#   * SinhSymTP's params.bScale
SymTP_bScale    = 0.5 # If SymTP chosen

# Step 1.d: Set the order of spatial and temporal derivatives;
#           the core data type, and the CFL factor.
# RK_method choices include: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
RK_method = "RK4"
FD_order  = 8        # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "double" # Best to use double here.
CFL_FACTOR= 0.1 #(GETS OVERWRITTEN WHEN EXECUTED.) In pure axisymmetry (symmetry_axes = 2 below) 1.0 works fine. Otherwise 0.5 or lower.

# Step 2.b: Set the timestepping order,
#           the core data type, and the CFL factor.
# Step 2.b: Set the order of spatial and temporal derivatives;
#           the core data type, and the CFL factor.
# RK_method choices include: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
#RK_method = "RK4"
#REAL      = "double" # Best to use double here.
#CFL_FACTOR = 0.1     # (GETS OVERWRITTEN WHEN EXECUTED.) In pure axisymmetry (symmetry_axes = 2 below) 1.0 works fine. Otherwise 0.5 or lower.

# Set the lapse & shift conditions
LapseCondition  = "OnePlusLog"
ShiftCondition  = "GammaDriving2ndOrder_Covariant"

# Step 4: Set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem",CoordSystem)
rfm.reference_metric() # Create ReU, ReDD needed for rescaling B-L initial data, generating BSSN RHSs, etc.

# Step 5: Set the finite differencing order to FD_order (set above).
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

# Step 6: Copy SIMD/SIMD_intrinsics.h to $Ccodesdir/SIMD/SIMD_intrinsics.h
#cmd.mkdir(os.path.join(Ccodesdir,"SIMD"))
#shutil.copy(os.path.join("nrpytutorial/SIMD/")+"SIMD_intrinsics.h",os.path.join(Ccodesdir,"SIMD/"))

# Step 7: Impose spherical symmetry by demanding that all
#         derivatives in the angular directions vanish
par.set_parval_from_str("indexedexp::symmetry_axes","12")

<a id='bssn'></a>

# Step 4: Output C code for BSSN spacetime solve \[Back to [top](#toc)\]
$$\label{bssn}$$

<a id='bssnrhs'></a>

## Step 4.a: Set up the BSSN and ScalarField right-hand-side (RHS) expressions, and add the *rescaled* $T^{\mu\nu}$ source terms \[Back to [top](#toc)\]
$$\label{bssnrhs}$$

`BSSN.BSSN_RHSs()` sets up the RHSs assuming a spacetime vacuum: $T^{\mu\nu}=0$. (This might seem weird, but remember that, for example, *spacetimes containing only single or binary black holes are vacuum spacetimes*.) Here, using the [`BSSN.BSSN_stress_energy_source_terms`](../edit/BSSN/BSSN_stress_energy_source_terms.py) ([**tutorial**](Tutorial-BSSN_stress_energy_source_terms.ipynb)) NRPy+ module, we add the $T^{\mu\nu}$ source terms to these equations.

In [ ]:
shutil.copytree("nrpytutorial/BSSN","BSSN")
shutil.copytree("nrpytutorial/ScalarField","ScalarField")

import time
import BSSN.BSSN_RHSs as rhs
import BSSN.BSSN_gauge_RHSs as gaugerhs
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::LapseEvolutionOption", LapseCondition)
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::ShiftEvolutionOption", ShiftCondition)

In [ ]:
from outputC import outC_function_dict
print(outC_function_dict.keys())
start = time.time()
# Enable rfm_precompute infrastructure, which results in
#   BSSN RHSs that are free of transcendental functions,
#   even in curvilinear coordinates, so long as
#   ConformalFactor is set to "W" (default).
cmd.mkdir(os.path.join(Ccodesdir,"rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute","True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir",os.path.join(Ccodesdir,"rfm_files/"))

# Evaluate BSSN + BSSN gauge RHSs with rfm_precompute enabled:
import BSSN.BSSN_quantities as Bq
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic","True")

rhs.BSSN_RHSs()

# Evaluate the Scalar Field RHSs
import ScalarField.ScalarField_RHSs as sfrhs
sfrhs.ScalarField_RHSs()

# Compute ScalarField T^{\mu\nu}
# Compute the scalar field energy-momentum tensor
import ScalarField.ScalarField_Tmunu as sfTmunu
sfTmunu.ScalarField_Tmunu()
T4UU = sfTmunu.T4UU

import BSSN.BSSN_stress_energy_source_terms as Bsest
Bsest.BSSN_source_terms_for_BSSN_RHSs(T4UU)
rhs.trK_rhs += Bsest.sourceterm_trK_rhs
for i in range(DIM):
    # Needed for Gamma-driving shift RHSs:
    rhs.Lambdabar_rhsU[i] += Bsest.sourceterm_Lambdabar_rhsU[i]
    # Needed for BSSN RHSs:
    rhs.lambda_rhsU[i]    += Bsest.sourceterm_lambda_rhsU[i]
    for j in range(DIM):
        rhs.a_rhsDD[i][j] += Bsest.sourceterm_a_rhsDD[i][j]
        
gaugerhs.BSSN_gauge_RHSs()

# We use betaU as our upwinding control vector:
Bq.BSSN_basic_tensors()
betaU = Bq.betaU

import BSSN.Enforce_Detgammahat_Constraint as EGC
enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammahat_Constraint_symb_expressions()

# Next compute Ricci tensor
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic","False")
Bq.RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()

# Now register the Hamiltonian as a gridfunction.
H = gri.register_gridfunctions("AUX","H")

# Then define the Hamiltonian constraint and output the optimized C code.
import BSSN.BSSN_constraints as bssncon
bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)
Bsest.BSSN_source_terms_for_BSSN_constraints(T4UU)
bssncon.H += Bsest.sourceterm_H

# Add Kreiss-Oliger dissipation
diss_strength = par.Cparameters("REAL","ScalarFieldCollapse",["diss_strength"],0.1)

alpha_dKOD   = ixp.declarerank1("alpha_dKOD")
cf_dKOD      = ixp.declarerank1("cf_dKOD")
trK_dKOD     = ixp.declarerank1("trK_dKOD")
sf_dKOD      = ixp.declarerank1("sf_dKOD")
sfM_dKOD     = ixp.declarerank1("sfM_dKOD")
betU_dKOD    = ixp.declarerank2("betU_dKOD","nosym")
vetU_dKOD    = ixp.declarerank2("vetU_dKOD","nosym")
lambdaU_dKOD = ixp.declarerank2("lambdaU_dKOD","nosym")
aDD_dKOD     = ixp.declarerank3("aDD_dKOD","sym01")
hDD_dKOD     = ixp.declarerank3("hDD_dKOD","sym01")

for k in range(3):
    gaugerhs.alpha_rhs += diss_strength*alpha_dKOD[k]*rfm.ReU[k]
    rhs.cf_rhs         += diss_strength*   cf_dKOD[k]*rfm.ReU[k]
    rhs.trK_rhs        += diss_strength*  trK_dKOD[k]*rfm.ReU[k]
    sfrhs.sf_rhs       += diss_strength*   sf_dKOD[k]*rfm.ReU[k]
    sfrhs.sfM_rhs      += diss_strength*  sfM_dKOD[k]*rfm.ReU[k]
    for i in range(3):
        if "2ndOrder" in ShiftCondition:
            gaugerhs.bet_rhsU[i] += diss_strength*   betU_dKOD[i][k]*rfm.ReU[k]
        gaugerhs.vet_rhsU[i]     += diss_strength*   vetU_dKOD[i][k]*rfm.ReU[k]
        rhs.lambda_rhsU[i]       += diss_strength*lambdaU_dKOD[i][k]*rfm.ReU[k]
        for j in range(3):
            rhs.a_rhsDD[i][j] += diss_strength*aDD_dKOD[i][j][k]*rfm.ReU[k]
            rhs.h_rhsDD[i][j] += diss_strength*hDD_dKOD[i][j][k]*rfm.ReU[k]
            
# Now that we are finished with all the rfm hatted
#           quantities in generic precomputed functional
#           form, let's restore them to their closed-
#           form expressions.
#par.set_parval_from_str("reference_metric::enable_rfm_precompute","False") # Reset to False to disable rfm_precompute.
if enable_rfm_precompute:
    par.set_parval_from_str("reference_metric::enable_rfm_precompute", "False") # Reset to False to disable rfm_precompute.
rfm.ref_metric__hatted_quantities()


end = time.time()
print("(BENCH) Finished BSSN symbolic expressions in "+str(end-start)+" seconds.")

def add_to_Cfunction_dict_BSSN_plus_ScalarField_RHSs():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"
    print("Generating C code for BSSN RHSs in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()

    # Construct the left-hand sides and right-hand-side expressions for all BSSN RHSs
    lhs_names = [        "alpha",       "cf",       "trK",         "sf",         "sfM"   ]
    rhs_exprs = [gaugerhs.alpha_rhs, rhs.cf_rhs, rhs.trK_rhs, sfrhs.sf_rhs, sfrhs.sfM_rhs]
    for i in range(3):
        lhs_names.append(        "betU"+str(i))
        rhs_exprs.append(gaugerhs.bet_rhsU[i])
        lhs_names.append(   "lambdaU"+str(i))
        rhs_exprs.append(rhs.lambda_rhsU[i])
        lhs_names.append(        "vetU"+str(i))
        rhs_exprs.append(gaugerhs.vet_rhsU[i])
        for j in range(i,3):
            lhs_names.append(   "aDD"+str(i)+str(j))
            rhs_exprs.append(rhs.a_rhsDD[i][j])
            lhs_names.append(   "hDD"+str(i)+str(j))
            rhs_exprs.append(rhs.h_rhsDD[i][j])

    # Sort the lhss list alphabetically, and rhss to match.
    #   This ensures the RHSs are evaluated in the same order
    #   they're allocated in memory:
    lhs_names,rhs_exprs = [list(x) for x in zip(*sorted(zip(lhs_names,rhs_exprs), key=lambda pair: pair[0]))]

    # Declare the list of lhrh's
    BSSN_evol_rhss = []
    for var in range(len(lhs_names)):
        BSSN_evol_rhss.append(lhrh(lhs=gri.gfaccess("rhs_gfs",lhs_names[var]),rhs=rhs_exprs[var]))

    # Set up the C function for the BSSN RHSs
    desc="Evaluate the BSSN RHSs"
    name="rhs_eval"
    add_to_Cfunction_dict(
        includes = includes,
        desc=desc,
        c_type = c_type, name = name,
        params   = """rfm_struct *restrict rfmstruct,const paramstruct *restrict params,
                      const REAL *restrict auxevol_gfs,const REAL *restrict in_gfs,REAL *restrict rhs_gfs""",
        body     = fin.FD_outputC("returnstring",BSSN_evol_rhss, params="outCverbose=False,enable_SIMD=True",
                                  upwindcontrolvec=betaU),   
        rel_path_to_Cparams=os.path.join("."),
        loopopts = "InteriorPoints,enable_SIMD,enable_rfm_precompute")
    end = time.time()
    print("(BENCH) Finished BSSN_RHS C codegen in " + str(end - start) + " seconds.")

def add_to_Cfunction_dict_Ricci():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    print("Generating C code for Ricci tensor in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()
    desc="Evaluate the Ricci tensor"
    c_type = "void"
    name="Ricci_eval"
    outfile  = os.path.join(Ccodesdir,name+".h") 
    desc=desc
    name=name
    params   = """rfm_struct *restrict rfmstruct,const paramstruct *restrict params,
                      const REAL *restrict in_gfs,REAL *restrict auxevol_gfs"""
    body     = fin.FD_outputC("returnstring",
                              [lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD00"),rhs=Bq.RbarDD[0][0]),
                                lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD01"),rhs=Bq.RbarDD[0][1]),
                                lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD02"),rhs=Bq.RbarDD[0][2]),
                                lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD11"),rhs=Bq.RbarDD[1][1]),
                                lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD12"),rhs=Bq.RbarDD[1][2]),
                                lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD22"),rhs=Bq.RbarDD[2][2])],
                                params="outCverbose=False,enable_SIMD=True")
    loopopts = "InteriorPoints,enable_SIMD,enable_rfm_precompute"
    end = time.time()
    print("(BENCH) Finished Ricci C codegen in " + str(end - start) + " seconds.")
    add_to_Cfunction_dict(
        includes = includes,
        desc = desc,
        c_type = c_type, name = name, params = params,
        body = body,
        rel_path_to_Cparams = os.path.join("."),
        loopopts = loopopts)



In [ ]:
# Step 6: Copy SIMD/SIMD_intrinsics.h to $Ccodesdir/SIMD/SIMD_intrinsics.h
cmd.mkdir(os.path.join(Ccodesdir,"SIMD"))
shutil.copy(os.path.join("nrpytutorial/SIMD/")+"SIMD_intrinsics.h",os.path.join(Ccodesdir,"SIMD/"))

In [ ]:
par.set_parval_from_str("finite_difference::enable_FD_functions", enable_FD_functions)

<a id='mol'></a>

## Step 1.b: Generate Method of Lines timestepping code \[Back to [top](#toc)\]
$$\label{mol}$$

The Method of Lines algorithm is described in detail in the [**NRPy+ tutorial notebook on Method of Lines algorithm**](Tutorial-Method_of_Lines-C_Code_Generation.ipynb).

In [ ]:
# Step 9: Generate Runge-Kutta-based (RK-based) timestepping code.
#       As described above the Table of Contents, this is a 2-step process:
#       9.b.A: Evaluate RHSs (RHS_string)
#       9.b.B: Apply boundary conditions (post_RHS_string, pt 1)
import MoLtimestepping.MoL_new_way as MoL
# from MoLtimestepping.RK_Butcher_Table_Dictionary import Butcher_dict
# RK_order  = Butcher_dict[RK_method][1]
RHS_string = """
Ricci_eval(&rfmstruct, &params, RK_INPUT_GFS, auxevol_gfs);
rhs_eval(&rfmstruct, &params, auxevol_gfs, RK_INPUT_GFS, RK_OUTPUT_GFS);"""

if OuterBoundaryCondition == "Sommerfeld":
    # Sommerfeld BCs are applied to the gridfunction RHSs directly
    RHS_string += "apply_bcs_sommerfeld(params, griddata->xx, bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_INPUT_GFS, RK_OUTPUT_GFS);"
    post_RHS_string = """
    enforce_detgammahat_constraint(&rfmstruct, &params,                     RK_OUTPUT_GFS);\n
    """
else:
    print("Invalid choice of boundary condition")
    sys.exit(1)

MoL.register_C_functions_and_NRPy_basic_defines(RK_method,
        RHS_string=RHS_string, post_RHS_string=post_RHS_string,
        enable_rfm=enable_rfm_precompute, enable_curviBCs=True, enable_SIMD=enable_SIMD, enable_griddata=True)

<a id='boundaryconditions'></a>

## Step 1.b: Output needed C code for boundary condition driver \[Back to [top](#toc)\]
$$\label{boundaryconditions}$$

In [ ]:
import CurviBoundaryConditions.CurviBoundaryConditions_new_way as CBC
CBC.CurviBoundaryConditions_register_C_functions_and_NRPy_basic_defines()
CBC.add_to_Cfunction_dict_set_up__bc_gz_map_and_parity_condns()

In [ ]:
import CurviBoundaryConditions.CurviBoundaryConditions as cbcs
if OuterBoundaryCondition == "Sommerfeld":
    bcs = cbcs.sommerfeld_boundary_condition_class(fd_order=6,
                                                 vars_radial_falloff_power_default=3,
                                                 vars_speed_default=1.0,
                                                 vars_at_inf_default=0.)
    bcs.write_sommerfeld_file(Ccodesdir)

<a id='cparams_rfm_and_domainsize'></a>

## Step 1.c: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]
$$\label{cparams_rfm_and_domainsize}$$

Here we output `free_parameters.h`, which sets initial data parameters, as well as grid domain & reference metric parameters, applying `domain_size` and `sinh_width`/`SymTP_bScale` (if applicable) as set above

In [ ]:
### UNSURE WHICH VERSION TO USE, NRPYElliptic and NRPy+ ARE PRETTY DIFFERENT HERE
### THIS CELL CONTAINS NRPyELLIPTIC

# Step 1.c.i: Set free_parameters.h
outstr = r"""
// Free parameters related to physical system:
params.time = 0.0; // Initial simulation time corresponds to time=0.

// Free parameters related to numerical timestep:
REAL CFL_FACTOR = """+str(CFL_FACTOR)+";\n"

# Append to $Ccodesrootdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above.
outstr += rfm.out_default_free_parameters_for_rfm("returnstring",
                                                  domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)

##WE'RE NOT USING THE PUNCTURE, 
# Append to $Ccodesrootdir/free_parameters.h puncture-related parameters
#outstr += r"""
#// Free parameters related to punctures
#"""
#for param, value in params_dict.items():
#    outstr += f"params.{param} = {value};\n"

with open(os.path.join(Ccodesdir,"free_parameters.h"),"w") as file:
    file.write(outstr.replace("params.", "griddata.params."))

In [ ]:
### THIS CELL CONTAINS NRPy+


# Step 4.e.i: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
#par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

# Step 4.e.ii: Set free_parameters.h
# Output to $Ccodesdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above.
#rfm.out_default_free_parameters_for_rfm(os.path.join(Ccodesdir,"free_parameters.h"),
#                                        domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)

# Step 4.e.iii: Generate set_Nxx_dxx_invdx_params__and__xx.h:
#rfm.set_Nxx_dxx_invdx_params__and__xx_h(Ccodesdir)

# Step 4.e.iv: Generate xx_to_Cart.h, which contains xx_to_Cart() for
#               (the mapping from xx->Cartesian) for the chosen
#               CoordSystem:
#rfm.xx_to_Cart_h("xx_to_Cart","./set_Cparameters.h",os.path.join(Ccodesdir,"xx_to_Cart.h"))

# Step 4.e.v: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
#par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

<a id='cfl'></a>

## Step 1.d: Output needed C code for finding the minimum proper distance between grid points, needed for [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673)-limited timestep \[Back to [top](#toc)\]
$$\label{cfl}$$

In order for our explicit-timestepping numerical solution to the scalar wave equation to be stable, it must satisfy the [CFL](https://en.wikipedia.org/w/index.php?title=Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition&oldid=806430673) condition:
$$
\Delta t \le \frac{\min(ds_i)}{c},
$$
where $c$ is the wavespeed, and
$$ds_i = h_i \Delta x^i$$ 
is the proper distance between neighboring gridpoints in the $i$th direction (in 3D, there are 3 directions), $h_i$ is the $i$th reference metric scale factor, and $\Delta x^i$ is the uniform grid spacing in the $i$th direction:

In [ ]:
##THIS IS NRPyELLIPTIC. I THINK WE NEED TO USE 
## NRPyELLIPTIC BECAUSE THIS IS HOW xx->cartesian IS GENERATED

# Generate & register C function set_Nxx_dxx_invdx_params__and__xx()
# Generate & register C function xx_to_Cart() for
#               (the mapping from xx->Cartesian) for the chosen
#               CoordSystem:
# Generate & register the find_timestep() function
rfm.register_C_functions(enable_rfm_precompute=enable_rfm_precompute, use_unit_wavespeed_for_find_timestep=True)
rfm.register_NRPy_basic_defines(enable_rfm_precompute=enable_rfm_precompute)

#from outputC import outC_function_dict
#print(outC_function_dict["find_timestep"])

In [ ]:
##THIS CELL CONTAINS NRPy+
# Output the find_timestep() function to a C file.
#rfm.out_timestep_func_to_file(os.path.join(Ccodesdir,"find_timestep.h"))

<a id='initial_data'></a>

# Step 2: Set up ADM Initial Conditions for the Scalar Field \[Back to [top](#toc)\]
$$\label{initial_data}$$

As documented [in the scalar field Gaussian pulse initial data NRPy+ tutorial notebook](nrpytutorial/Tutorial-ADM_Initial_Data-ScalarField.ipynb), we will now set up the scalar field initial data, storing the densely-sampled result to file.

The initial data function `ScalarField_InitialData` requires `SciPy`, so let's make sure it's installed.

This portion of the code is not in NRPyElliptic tutorial, which makes sense. The initial conditions for NRPyElliptic are generated differently, and the physical scenario in NRPyElliptic is a different one (a wave).

In [ ]:
# Step 2.a: Import necessary Python and NRPy+ modules
import ScalarField.ScalarField_InitialData as sfid

# Step 2.b: Set the initial data parameters
outputfilename  = os.path.join(outdir,"SFID.txt")
ID_Family       = "Gaussian_pulse"
pulse_amplitude = 0.4
pulse_center    = 0
pulse_width     = 1
Nr              = 30000
rmax            = domain_size*1.1

# Step 2.c: Generate the initial data #Generates a text file so it won't work with emscripten. Leo is making C code.
sfid.ScalarField_InitialData(outputfilename,ID_Family,
                             pulse_amplitude,pulse_center,pulse_width,Nr,rmax)

# Step 2.d: Generate the needed C code
sfid.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(Ccodesdir=Ccodesdir)

<a id='ADMtoBSSN'></a>

# Step 3: Convert ADM to BSSN Coordinates \[Back to [top](#toc)\]
$$\label{ADMtoBSSN}$$

In [ ]:
import BSSN.ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear as AtoBnum
AtoBnum.Convert_Spherical_or_Cartesian_ADM_to_BSSN_curvilinear("Spherical","ID_scalarfield_ADM_quantities",
                                                               Ccodesdir=Ccodesdir,loopopts="")

<a id='hamconstraint'></a>

## Step 4.b: Output C code for Hamiltonian constraint \[Back to [top](#toc)\]
$$\label{hamconstraint}$$

Next output the C code for evaluating the Hamiltonian constraint [(**Tutorial**)](Tutorial-BSSN_constraints.ipynb). In the absence of numerical error, this constraint should evaluate to zero. However it does not due to numerical (typically truncation and roundoff) error. We will therefore measure the Hamiltonian constraint violation to gauge the accuracy of our simulation, and, ultimately determine whether errors are dominated by numerical finite differencing (truncation) error as expected.

In [ ]:
def add_to_Cfunction_dict_Hamiltonian():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"
    start = time.time()
    print("Generating optimized C code for Hamiltonian constraint. May take a while, depending on CoordSystem.")
    # Set up the C function for the Hamiltonian RHS
    desc="Evaluate the Hamiltonian constraint"
    name="Hamiltonian_constraint"
    params   = """rfm_struct *restrict rfmstruct,const paramstruct *restrict params,
                      REAL *restrict in_gfs, REAL *restrict auxevol_gfs, REAL *restrict aux_gfs"""
    body     = fin.FD_outputC("returnstring",lhrh(lhs=gri.gfaccess("aux_gfs", "H"), rhs=bssncon.H),
                                  params="outCverbose=False")
    loopopts = "InteriorPoints,enable_rfm_precompute"

    add_to_Cfunction_dict(
    includes = includes,
    desc = desc,
    c_type = c_type, name = name, params = params,
    body = body,
    rel_path_to_Cparams = os.path.join("."),
    loopopts = loopopts)
    end = time.time()
    print("(BENCH) Finished Hamiltonian C codegen in " + str(end - start) + " seconds.")

<a id='enforce3metric'></a>

## Step 4.c: Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint \[Back to [top](#toc)\]
$$\label{enforce3metric}$$

Then enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (Eq. 53 of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)), as [documented in the corresponding NRPy+ tutorial notebook](nrpytutorial/Tutorial-BSSN_enforcing_determinant_gammabar_equals_gammahat_constraint.ipynb)

Applying curvilinear boundary conditions should affect the initial data at the outer boundary, and will in general cause the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint to be violated there. Thus after we apply these boundary conditions, we must always call the routine for enforcing the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint:

In [ ]:
## This function outputs the C script automatically, so 
## it doesn't need to be added to the dictionary.
#def gammadet():
#    start = time.time()
#    print("Generating optimized C code for gamma constraint. May take a while, depending on CoordSystem.")

    # Set up the C function for the det(gammahat) = det(gammahat)
#    EGC.output_Enforce_Detgammahat_Constraint_Ccode(Ccodesdir,exprs=enforce_detg_constraint_symb_expressions)
#    end = time.time()
#    print("(BENCH) Finished gamma constraint C codegen in " + str(end - start) + " seconds.")

## TAKEN FROM THE BSSN file enforce_detgammahat_constraint.py
def add_to_Cfunction_dict_gammadet():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"
    desc = "Enforce det(gammabar) = det(gammahat) constraint."
    name = "enforce_detgammahat_constraint"
    params = "const rfm_struct *restrict rfmstruct, const paramstruct *restrict params, REAL *restrict in_gfs"
    loopopts = "AllPoints,enable_rfm_precompute"
    body=fin.FD_outputC("returnstring", enforce_detg_constraint_symb_expressions,
                        params="outCverbose=False,preindent=1,includebraces=False")
    add_to_Cfunction_dict(
    includes = includes,
    desc = desc,
    c_type = c_type, name = name, params = params,
    body = body,
    rel_path_to_Cparams = os.path.join("."),
    loopopts = loopopts)

<a id='main_ccode'></a>

# Step 6: `ScalarFieldCollapse_Playground.c`: The Main C Code \[Back to [top](#toc)\]

$$\label{main_ccode}$$

## Main c files

In [ ]:
# Part P0: Define REAL, set the number of ghost cells NGHOSTS (from NRPy+'s FD_CENTDERIVS_ORDER),
#          and set the CFL_FACTOR (which can be overwritten at the command line)

#with open(os.path.join(Ccodesdir,"ScalarFieldCollapse_Playground_REAL__NGHOSTS__CFL_FACTOR.h"), "w") as file:
#    file.write("""
#// Part P0.a: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
#define NGHOSTS """+str(int(FD_order/2)+1)+"""
#// Part P0.b: Set the numerical precision (REAL) to double, ensuring all floating point
#//            numbers are stored to at least ~16 significant digits
#define REAL """+REAL+"""
#// Part P0.c: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
#REAL CFL_FACTOR = """+str(CFL_FACTOR)+"""; // Set the CFL Factor. Can be overwritten at command line.\n""")

def add_to_Cfunction_dict_ScalarFieldCollapse_Playground_REAL__NGHOSTS__CFL_FACTOR():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    c_type = "void"
    name = "define_REAL__NGHOSTS__CFL"
    desc = """Define REAL, set the number of ghost cells NGHOSTS (from NRPy+'s FD_CENTDERIVS_ORDER', 
    and set the CFL_FACTOR (can be overwritten at command line)
    """
    params = ""
    body = """
    // Part P0.a: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
    #define NGHOSTS """+str(int(FD_order/2)+1)+"""
    // Part P0.b: Set the numerical precision (REAL) to double, ensuring all floating point
    //            numbers are stored to at least ~16 significant digits
    #define REAL """+REAL+"""
    // Part P0.c: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
    REAL CFL_FACTOR = """+str(CFL_FACTOR)+"""; // Set the CFL Factor. Can be overwritten at command line.\n"""

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."))

In [ ]:
def add_to_Cfunction_dict_SommerCrit_Playground():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h", "<sys/stat.h>"]
    c_type = "int"
    name = "main"
    desc = "make field go flat"
    params = ""
    
    body = r"""//This is being written into a C-file 
    //Emscripten compiles the C code
    // Step P0: Define REAL and NGHOSTS; and declare CFL_FACTOR. This header is generated in NRPy+.
    #include "ScalarFieldCollapse_Playground_REAL__NGHOSTS__CFL_FACTOR.h"

    #include "rfm_files/rfm_struct__declare.h"

    #include "declare_Cparameters_struct.h"

    #include "emscripten.h"

    // All SIMD intrinsics used in SIMD-enabled C code loops are defined here:
    #include "SIMD/SIMD_intrinsics.h"

    // Step P1: Import needed header files
    #include "stdio.h"
    #include "stdlib.h"
    #include "math.h"
    #include "time.h"
    #include "stdint.h" // Needed for Windows GCC 6.x compatibility
    #ifndef M_PI
    #define M_PI 3.141592653589793238462643383279502884L
    #endif
    #ifndef M_SQRT1_2
    #define M_SQRT1_2 0.707106781186547524400844362104849039L
    #endif
    #define wavespeed 1.0 // Set CFL-based "wavespeed" to 1.0.
    #define alpha_threshold (2e-3) // Value below which we rule gravitational collapse has happened

    // Step P2: Declare the IDX4S(gf,i,j,k) macro, which enables us to store 4-dimensions of
    //           data in a 1D array. In this case, consecutive values of "i"
    //           (all other indices held to a fixed value) are consecutive in memory, where
    //           consecutive values of "j" (fixing all other indices) are separated by
    //           Nxx_plus_2NGHOSTS0 elements in memory. Similarly, consecutive values of
    //           "k" are separated by Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1 in memory, etc.
    #define IDX4S(g,i,j,k) \
    ( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )
    #define IDX4ptS(g,idx) ( (idx) + (Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2) * (g) )
    #define IDX3S(i,j,k) ( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) ) ) )
    #define LOOP_REGION(i0min,i0max, i1min,i1max, i2min,i2max) \
      for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++)
    #define LOOP_ALL_GFS_GPS(ii) _Pragma("omp parallel for") \
      for(int (ii)=0;(ii)<Nxx_plus_2NGHOSTS_tot*NUM_EVOL_GFS;(ii)++)

    // Step P3: Set UUGF and VVGF macros, as well as xx_to_Cart()
    #include "boundary_conditions/gridfunction_defines.h"

    // Step P4: Set xx_to_Cart(const paramstruct *restrict params,
    //                     REAL *restrict xx[3],
    //                     const int i0,const int i1,const int i2,
    //                     REAL xCart[3]),
    //           which maps xx->Cartesian via
    //    {xx[0][i0],xx[1][i1],xx[2][i2]}->{xCart[0],xCart[1],xCart[2]}
    #include "xx_to_Cart.h"

    // Step P5: Defines set_Nxx_dxx_invdx_params__and__xx(const int EigenCoord, const int Nxx[3],
    //                                       paramstruct *restrict params, REAL *restrict xx[3]),
    //          which sets params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for
    //          the chosen Eigen-CoordSystem if EigenCoord==1, or
    //          CoordSystem if EigenCoord==0.
    #include "set_Nxx_dxx_invdx_params__and__xx.h"

    // Step P6: Include basic functions needed to impose curvilinear
    //          parity and boundary conditions.
    #include "boundary_conditions/CurviBC_include_Cfunctions.h"

    // Step P7: Implement the algorithm for upwinding.
    //          *NOTE*: This upwinding is backwards from
    //          usual upwinding algorithms, because the
    //          upwinding control vector in BSSN (the shift)
    //          acts like a *negative* velocity.
    //#define UPWIND_ALG(UpwindVecU) UpwindVecU > 0.0 ? 1.0 : 0.0

    // Step P8: Include function for enforcing detgammahat constraint.
    #include "enforce_detgammahat_constraint.h"

    // Step P9: Find the CFL-constrained timestep
    #include "find_timestep.h"

    // Step P10: Declare initial data input struct:
    //           stores data from initial data solver,
    //           so they can be put on the numerical grid.
    typedef struct __ID_inputs {
        int interp_stencil_size;
        int numlines_in_file;
        REAL *r_arr,*sf_arr,*psi4_arr,*alpha_arr;
    } ID_inputs;

    // Part P11: Declare all functions for setting up ScalarField initial data.
    /* Routines to interpolate the ScalarField solution and convert to ADM & T^{munu}: */
    #include "../nrpytutorial/ScalarField/ScalarField_interp.h"
    #include "ID_scalarfield_ADM_quantities.h"
    #include "ID_scalarfield_spherical.h"
    #include "ID_scalarfield_xx0xx1xx2_to_BSSN_xx0xx1xx2.h"
    #include "ID_scalarfield.h"

    /* Next perform the basis conversion and compute all needed BSSN quantities */
    #include "ID_ADM_xx0xx1xx2_to_BSSN_xx0xx1xx2__ALL_BUT_LAMBDAs.h"
    #include "ID_BSSN__ALL_BUT_LAMBDAs.h"
    #include "ID_BSSN_lambdas.h"



    float fr[30000];
    float fsf[30000];
    float fpsi4[30000];
    float falpha[30000];


    // Step P12: Set the generic driver function for setting up BSSN initial data
    void initial_data(const paramstruct *restrict params,const bc_struct *restrict bcstruct,
                      const rfm_struct *restrict rfmstruct,
                      REAL *restrict xx[3], REAL *restrict auxevol_gfs, REAL *restrict in_gfs) {
    #include "set_Cparameters.h"
    //We will change this to getter functions to grab the initial data from Leos initial conditions
        // Step 1: Set up ScalarField initial data
        // Step 1.a: Read ScalarField initial data from data file
        // Open the data file:
        // [Deleted to hard code in initial conditions]
        // Count the number of lines in the data file:
    
    
        // Allocate space for all data arrays:
        int numlines_in_file = 30000;
        REAL *r_arr     = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
        REAL *sf_arr    = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
        REAL *psi4_arr  = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
        REAL *alpha_arr = (REAL *)malloc(sizeof(REAL)*numlines_in_file);

        // Read from the data file, filling in arrays
        // read_datafile__set_arrays() may be found in ScalarField/ScalarField_interp.h
        #include "../SFID.h"
        //REAL *r_arr = r_arrjs;
        //REAL *sf_arr = sf_arrjs;
        //REAL *psi4_arr = psi4_arrjs;
        //REAL *alpha_arr = alpha_arrjs;
    
        //fprintf(stderr,"R_arr %e");
    
        for(int ln =0; ln<numlines_in_file;ln++){
            r_arr[ln] = fr[ln];
            sf_arr[ln] = fsf[ln];
            psi4_arr[ln] = fpsi4[ln];
            alpha_arr[ln] = falpha[ln];
        }
        //fprintf(stderr,"INITIAL_DATA %e %e %e %e TEST\n", r_arr[29999], sf_arr[29999], psi4_arr[29999],alpha_arr[29999]);
    
        const int interp_stencil_size = 12;
        ID_inputs SF_in;
        SF_in.interp_stencil_size = interp_stencil_size;
        SF_in.numlines_in_file    = numlines_in_file;
        SF_in.r_arr               = r_arr;
        SF_in.sf_arr              = sf_arr;
        SF_in.psi4_arr            = psi4_arr;
        SF_in.alpha_arr           = alpha_arr;

        // Step 1.b: Interpolate data from data file to set BSSN gridfunctions
        ID_scalarfield(params,xx,SF_in, in_gfs);
        ID_BSSN__ALL_BUT_LAMBDAs(params,xx,SF_in, in_gfs);
        apply_bcs_curvilinear(params, bcstruct, NUM_EVOL_GFS, evol_gf_parity, in_gfs);
        enforce_detgammahat_constraint(rfmstruct, params,                   in_gfs);
        ID_BSSN_lambdas(params, xx, in_gfs);
        apply_bcs_curvilinear(params, bcstruct, NUM_EVOL_GFS, evol_gf_parity, in_gfs);
        enforce_detgammahat_constraint(rfmstruct, params,                   in_gfs);

        free(r_arr);
        free(sf_arr);
        free(psi4_arr);
        free(alpha_arr);
    }

    // Step P11: Declare function for evaluating Hamiltonian constraint (diagnostic)
    #include "Hamiltonian_constraint.h"

    // Step P12: Declare rhs_eval function, which evaluates BSSN RHSs
    #include "rhs_eval.h"

    // Step P13: Declare Ricci_eval function, which evaluates Ricci tensor
    #include "Ricci_eval.h"

    //#include "NRPyCritCol_regridding.h"

    REAL rho_max = 0.0;
    //This is where the code diverges from the tutorial.
    // (These used to be defined in main function, but to make them global they were moved outside main, and getter/setter 
    // functions are used to modify them)
    // Step P15: Declare global pointers and variables to be referenced in getter function.
    //           This must be done in order to access variables created in the initialize
    //           function in the stepfoward function.
    int dimensions[4] = {28, 8, 2, 18}; //#Filler values that are updated later on

    
    int arrNGHOSTS[4];
    paramstruct params_p;
    rfm_struct rfmstruct_p;
    bc_struct bcstruct_p;
    REAL N_final_p, output_every_N_p, dt_p;
    //Contains scalar & metric fields, but also psi_4, so dont need the psi4 ones.
    // Make resolution + cfl factor global variables. Generally make most variables global. 
    REAL *y_n_gfs_p, *auxevol_gfs_p, *k_odd_gfs_p, *k_even_gfs_p, *y_nplus1_running_total_gfs_p, *xx_p[3];

    // initialize() function:
    // Step 0: Set up grid structure, allocate memory for gridfunctions, set up coordinates
    // Step 1: Set up initial data to an exact solutiondef
    // Step 2: Initialize global pointers and variables

    // Main function replaced by initialize and getter functions. Emscripten connects to Javascript.
    int EMSCRIPTEN_KEEPALIVE initialize(REAL CFL_FACTOR){
    
        //fprintf(stderr,"Running initialize!!\n");
        paramstruct params;
    #include "set_Cparameters_default.h"

        // Step 0a: Set up numerical grid structure, first in space...
        int n1 = dimensions[0];
        int n2 = dimensions[1];
        int n3 = dimensions[2];
        const int Nxx[3] = {n1,n2,n3};

        // Step 0b: Set free parameters, overwriting Cparameters defaults
        //          by hand or with command-line input, as desired.
    #include "free_parameters.h"

        // Step 0c: Uniform coordinate grids are stored to *xx[3]
        REAL *xx[3];
        // Step 0c.i: Set bcstruct
        bc_struct bcstruct;
        {
            int EigenCoord = 1;
            // Step 0c.ii: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
            //             params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
            //             chosen Eigen-CoordSystem.
        
            set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);
        
            // Step 0c.iii: Set Nxx_plus_2NGHOSTS_tot
    #include "set_Cparameters-nopointer.h"
            const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;
            // Step 0d: Find ghostzone mappings; set up bcstruct
    #include "boundary_conditions/driver_bcstruct.h"
            // Step 0d.i: Free allocated space for xx[][] array
            for(int i=0;i<3;i++) free(xx[i]);
        }
    
        // Step 0e: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
        //          params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
        //          chosen (non-Eigen) CoordSystem.
        int EigenCoord = 0;
        set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);

        // Step 0f: Set all C parameters "blah" for params.blah, including
        //          Nxx_plus_2NGHOSTS0 = params.Nxx_plus_2NGHOSTS0, etc.
        #include "set_Cparameters-nopointer.h"
        const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

        // Step 0g: Time coordinate parameters
        const REAL t_final = 16; // Run until the user says stop, but let them be able to pause and continue, with reset button.

        // Step 0h: Set timestep based on smallest proper distance between gridpoints and CFL factor
        REAL dt = find_timestep(&params, xx);
    
        N_final_p = (int)(t_final / dt + 0.5); // The number of points in time.
                                             // Add 0.5 to account for C rounding down
                                             // typecasts to integers.
        //The Collision Code:
        //REAL out_approx_every_t = 0.2;
        //output_every_N_p = (int)(out_approx_every_t*((REAL)N_final_p)/t_final);
        //The Collapse code is below:
        //Every Nth step writes it to a file. For our version, instead of writing to a file just 
        //            1. Stores data 2. animates
        // We probably wont need to store data.
        REAL output_every_N = 20;
    
    

        // Step 0i: Error out if the number of auxiliary gridfunctions outnumber evolved gridfunctions.
        //              This is a limitation of the RK method. You are always welcome to declare & allocate
        //              additional gridfunctions by hand.
    
        if(NUM_AUX_GFS > NUM_EVOL_GFS) {
            fprintf(stderr,"Error: NUM_AUX_GFS > NUM_EVOL_GFS. Either reduce the number of auxiliary gridfunctions,\n");
            fprintf(stderr,"       or allocate (malloc) by hand storage for *diagnostic_output_gfs. \n");
            exit(1);
        }

        // Step 0j: Allocate memory for gridfunctions
    #include "MoLtimestepping/RK_Allocate_Memory.h"
        REAL *restrict auxevol_gfs = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS_tot);

        // Step 0k: Set up precomputed reference metric arrays
        // Step 0k.i: Allocate space for precomputed reference metric arrays.
        rfm_struct rfmstruct;
        rfm_precompute_rfmstruct_malloc(&griddata.params, &griddata.rfmstruct);

        // Step 1a: Set up initial data to an exact solution
        initial_data(&params,&bcstruct, &rfmstruct, xx, auxevol_gfs, y_n_gfs);

        // Step 1b: Apply boundary conditions, as initial data
        //          are sometimes ill-defined in ghost zones.
        //          E.g., spherical initial data might not be
        //          properly defined at points where r=-1.
        apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS,evol_gf_parity, y_n_gfs);
        enforce_detgammahat_constraint(&rfmstruct, &params, y_n_gfs);

        //Step 2: Assign pointers/Initialize global variables
        arrNGHOSTS[0] = NGHOSTS;
        arrNGHOSTS[1] = Nxx_plus_2NGHOSTS0;
        arrNGHOSTS[2] = Nxx_plus_2NGHOSTS1;
        arrNGHOSTS[3] = Nxx_plus_2NGHOSTS2;
        dt_p = dt; 
        //These appear to be all the pointers needed 
        params_p = params;
        rfmstruct_p = rfmstruct;
        bcstruct_p = bcstruct;
        y_n_gfs_p = y_n_gfs;
        auxevol_gfs_p = auxevol_gfs;
        k_odd_gfs_p = k_odd_gfs;
        k_even_gfs_p = k_even_gfs;
        y_nplus1_running_total_gfs_p = y_nplus1_running_total_gfs;
        xx_p[0]=xx[0];
        xx_p[1]=xx[1];
        xx_p[2]=xx[2];
        //printf("%f",xx_p[0]);

        return 0;
    }

    // stepForward() function:
    // Step 1: Define and initialize variables from initialize() function so they can be used in the RK-like Method
    //         of Lines timestepping algorithm
    // Step 2: Step forward one timestep (t -> t+dt) in time using chosen RK-like MoL timestepping algorithm
    // Second half of main, does one single timescript that Javascript loops
    void EMSCRIPTEN_KEEPALIVE stepForward(j){
    
        // Step 1: Redefine and initialize variables. In order to call each time-step one by one, we need to redefine
        //         some variables used in the MoL timestepping algorithm with the saved values from the initialization
        //         step
        int Nxx_plus_2NGHOSTS0 = arrNGHOSTS[1];
        int Nxx_plus_2NGHOSTS1 = arrNGHOSTS[2];
        int Nxx_plus_2NGHOSTS2 = arrNGHOSTS[3];
        int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;
        int N_final = N_final_p;
        int output_every_N = output_every_N_p;
        //REAL *restrict diagnostic_output_gfs = diagnostic_output_gfs_p;
        REAL dt = dt_p;
        paramstruct params = params_p;
        rfm_struct rfmstruct = rfmstruct_p;
        bc_struct bcstruct = bcstruct_p;
        REAL *y_n_gfs = y_n_gfs_p;
        REAL *restrict auxevol_gfs = auxevol_gfs_p;
        REAL *k_odd_gfs = k_odd_gfs_p;
        REAL *k_even_gfs = k_even_gfs_p;
        REAL *restrict y_nplus1_running_total_gfs = y_nplus1_running_total_gfs_p;
        REAL *xx[3];
        xx[0]=xx_p[0];
        xx[1]=xx_p[1];
        xx[2]=xx_p[2];
        //if(j){
        //    size_t len = sizeof(y_n_gfs_p)/sizeof(y_n_gfs_p[0]);
        //    fprintf(stderr, "The length of the array is: %zu\n", len);
        //}
    #include "boundary_conditions/driver_bcstruct.h"

        // Step 2: Step forward one timestep (t -> t+dt) in time using
        //           chosen RK-like MoL timestepping algorithm
        // Step 3.a: Output 2D data file periodically, for visualization
      //  if(n%output_every_N == 0) {
            // Evaluate Hamiltonian constraint violation
       //     Hamiltonian_constraint(&rfmstruct, &params, y_n_gfs,auxevol_gfs, diagnostic_output_gfs);

            //char filename[100];
            //sprintf(filename,"out%d-%08d.txt",Nxx[0],n);
            //const int i1mid=Nxx_plus_2NGHOSTS1/2;
            //const int i2mid=Nxx_plus_2NGHOSTS2/2;
            //FILE *fp = fopen(filename, "w");
            //for( int i0=NGHOSTS;i0<Nxx_plus_2NGHOSTS0-NGHOSTS;i0++) {
            //    const int idx  = IDX3S(i0,i1mid,i2mid);
            //    const REAL xx0 = xx[0][i0];
            //    REAL xCart[3];
            //    xx_to_Cart(&params,xx,i0,i1mid,i2mid,xCart);
            //    const REAL rr = sqrt( xCart[0]*xCart[0] + xCart[1]*xCart[1] + xCart[2]*xCart[2] );
            //    fprintf(fp,"%e %e %e %e %e %e %e\n",xx0,rr,
            //            y_n_gfs[IDX4ptS(SFGF,idx)],y_n_gfs[IDX4ptS(SFMGF,idx)],
            //            y_n_gfs[IDX4ptS(ALPHAGF,idx)],y_n_gfs[IDX4ptS(CFGF,idx)],
            //            log10(fabs(diagnostic_output_gfs[IDX4ptS(HGF,idx)])));
            //}
            //fclose(fp);
        //}

    #include "MoLtimestepping/RK_MoL.h"
    }



    // Getter functions used to access the data in javascript after compliling.

    // getNFinal(): returns final time-step value
    REAL EMSCRIPTEN_KEEPALIVE getNFinal(){
        return N_final_p;
    }

    // setNFinal(): sets the final time-step value
    void EMSCRIPTEN_KEEPALIVE setNFinal(int i){
        N_final_p = i;
    }

    // getdt(): gets the value for dt
    REAL EMSCRIPTEN_KEEPALIVE getdt(){
        return dt_p;
    }

    // getNGHOSTS(): returns desired dimensions including ghosts shells
    REAL EMSCRIPTEN_KEEPALIVE getNGHOSTS(int i){
        return arrNGHOSTS[i];
    }

    // getFxVal(): returns desired function value at specfic point in space, index i is determined from the
    //             IDX3S and IDX4ptS arrays
    REAL EMSCRIPTEN_KEEPALIVE getFxVal(int i){
        return y_n_gfs_p[i];
    }

    // getIDX3S(): returns IDX3S index for a given point
    REAL EMSCRIPTEN_KEEPALIVE getIDX3S(int i0, int i1, int i2){
        int Nxx_plus_2NGHOSTS0 = arrNGHOSTS[1];
        int Nxx_plus_2NGHOSTS1 = arrNGHOSTS[2];
        int Nxx_plus_2NGHOSTS2 = arrNGHOSTS[3];
        return IDX3S(i0,i1,i2);
    }

    // getIDX4ptS(): returns IDX4ptS index to be used in getFxVal() using information from getIDX3S()
    REAL EMSCRIPTEN_KEEPALIVE getIDX4ptS(int fx, REAL idx){
        int Nxx_plus_2NGHOSTS0 = arrNGHOSTS[1];
        int Nxx_plus_2NGHOSTS1 = arrNGHOSTS[2];
        int Nxx_plus_2NGHOSTS2 = arrNGHOSTS[3];
        return IDX4ptS(fx,idx);
    }

    // getCart(): returns the cartesian version of a given spherical coordinate
    REAL EMSCRIPTEN_KEEPALIVE getCart(int i0, int i1, int i2, int index){
        REAL xCart[3];
        xx_to_Cart(&params_p,xx_p,i0,i1,i2,xCart);
        return xCart[index];
    }

    // setDim(): setter function allowing the user to change the resolution of the simulation
    void EMSCRIPTEN_KEEPALIVE setDim(int dim1, int dim2, int dim3){
        dimensions[0] = dim1;
        dimensions[1] = dim2;
        dimensions[2] = dim3;
        dimensions[3]= dim1/4;
    }

    // getDim(): getter function for the dimensions/resolution of the simulation
    int EMSCRIPTEN_KEEPALIVE getDim(int i){
        return dimensions[i];
    }

    // setInitials(): setter functions for the initial conditions
    void EMSCRIPTEN_KEEPALIVE setInitR(float i, int j){
        fr[j] = i;
    }
    void EMSCRIPTEN_KEEPALIVE setInitSf(float i, int j){
        fsf[j] = i;
    }
    void EMSCRIPTEN_KEEPALIVE setInitPsi4(float i, int j){
        fpsi4[j] = i;
    }
    void EMSCRIPTEN_KEEPALIVE setInitAlpha(float i, int j){
        falpha[j] = i;
    }
    // A very crude attempt that hopefully returns the scalar field values
    //void EMSCRIPTEN_KEEPALIVE getSf(int i){
    //    return sf[i];
    //} (It did not return the scalar field)

    // runsim(): function that runs the initialize function with a specific resolution and CFL factor
    void EMSCRIPTEN_KEEPALIVE runsim(){
        CFL_FACTOR = 1.0;
    
        fprintf(stderr,"Running Sim3!\n");
    
        initialize(CFL_FACTOR);
    
        fprintf(stderr,"Finished Sim!\n");
    }
    return 0;"""
    
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), enableCparameters=False)


In [ ]:
def register_C_code_functions_NRPyElliptic():
    add_to_Cfunction_dict_BSSN_plus_ScalarField_RHSs()
    add_to_Cfunction_dict_Ricci()
    add_to_Cfunction_dict_Hamiltonian()
    add_to_Cfunction_dict_gammadet()
    add_to_Cfunction_dict_ScalarFieldCollapse_Playground_REAL__NGHOSTS__CFL_FACTOR()
    add_to_Cfunction_dict_SommerCrit_Playground()

import outputC as outC
outC.outputC_register_C_functions_and_NRPy_basic_defines()  # #define M_PI,  etc.
# Declare paramstruct, register set_Cparameters_to_default(),
#   and output declare_Cparameters_struct.h and set_Cparameters[].h:
outC.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(os.path.join(Ccodesdir))

gri.register_C_functions_and_NRPy_basic_defines(enable_rfmstruct=enable_rfm_precompute)  # #define IDX3S(),  etc.
fin.register_C_functions_and_NRPy_basic_defines(NGHOSTS_account_for_onezone_upwind=False)  # #define NGHOSTS, etc.

# all functions needed for solver:
register_C_code_functions_NRPyElliptic()

# Output functions for computing all finite-difference stencils.
#   Must be called after defining all functions depending on FD stencils.
if enable_FD_functions:
    fin.output_finite_difference_functions_h(path=Ccodesdir)

# Call this last: Set up NRPy_basic_defines.h and NRPy_function_prototypes.h.
outC.construct_NRPy_basic_defines_h(Ccodesdir, enable_SIMD=enable_SIMD)
outC.construct_NRPy_function_prototypes_h(Ccodesdir)

shutil.rmtree("BSSN", ignore_errors=True)
shutil.rmtree("ScalarField",ignore_errors=True)

In [ ]:
# Needed for Sommerfeld boundary
with open(os.path.join(Ccodesdir,"NRPy_basic_defines.h"),"a") as file:
    file.write("static REAL evolgf_speed[NUM_EVOL_GFS];")

In [ ]:
from outputC import outC_function_dict
print(outC_function_dict.keys())

In [ ]:
import cmdline_helper as cmd
cmd.new_C_compile(Ccodesdir, "NRPyCritColl_Playground",
                  uses_free_parameters_h=True, compiler_opt_option="fast") # fastdebug or debug also supported
os.chdir(Ccodesdir)
#cmd.Execute("NRPyElliptic_Playground", f"{N0} {N1} {N2} 3000", os.path.join("output", "out-medresolution.txt"))
# Return to root directory
#os.chdir(os.path.join(".."))

Now that we have all the necessary c files, we can delete main and then recreate the original C file.

In [ ]:
os.remove(Ccodesdir+"main.c")
os.remove(Ccodesdir+"Makefile")

#Go into define_REAL__NGHOSTS__CFL.h and remove function definition / includes
# Comment out NGHOSTS, and replace it in NRPy_basic_defines.
os.rename(Ccodesdir+"define_REAL__NGHOSTS__CFL.c", Ccodesdir+"define_REAL__NGHOSTS__CFL.h")

#Remove CFL factor from free_parameters.h

#Go into ID_ADM_ID_ADM_xx0xx1xx2_to_BSSN_xx0xx1xx2__ALL_BUT_LAMBDAs 
# and add params-> to all the paramstruct members

#Go into ID_BSSN_lambdas.h
# and add params-> to all the paramstruct members

#Add time to paramstruct from NRPyElliptic

#Rename xxtocart.c to xxtocart.h, and remove NRPy_basic_defines.h and add static inline

#Rename set_Nxx.... to .h file, remove call to NRPy_basic_defines.h

#Rename find_timestep.c to .h, and remove call to NRPy_basic_defines.h

#Rename MoL_step_forward_in_time.c to .h, remove includes. 

#Rename set_Cparameters_to_default.c to .h, remove function + includes.

#Copy rfm_struct into NRPy_basic_defines.h and add to griddata_struct

#Make rhs_eval.c and Ricci_eval.c .h files

#In MoL_step_forward_in_time.c, make it .h, then add pointers for all rfm's
# Also had to make a copy of enforce_gammadet_constraint, because Idk how to
# call it so it's in Playground while also having it act in 
# MoL_step_forward_in_time without double declaring it.

In [136]:
%%writefile $Ccodesdir/ScalarFieldCollapse_Playground.c
//This is being written into a C-file 
//Emscripten compiles the C code
// Step P0: Define REAL and NGHOSTS; and declare CFL_FACTOR. This header is generated in NRPy+.
#include "define_REAL__NGHOSTS__CFL.h"

//#include "rfm_files/rfm_struct__declare.h"

//#include "declare_Cparameters_struct.h"
#include "NRPy_basic_defines.h"

#include "emscripten.h"

// All SIMD intrinsics used in SIMD-enabled C code loops are defined here:
#include "SIMD/SIMD_intrinsics.h"

// Step P1: Import needed header files
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "time.h"
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#ifndef M_PI
#define M_PI 3.141592653589793238462643383279502884L
#endif
#ifndef M_SQRT1_2
#define M_SQRT1_2 0.707106781186547524400844362104849039L
#endif
#define wavespeed 1.0 // Set CFL-based "wavespeed" to 1.0.
#define alpha_threshold (2e-3) // Value below which we rule gravitational collapse has happened

// Step P2: Declare the IDX4S(gf,i,j,k) macro, which enables us to store 4-dimensions of
//           data in a 1D array. In this case, consecutive values of "i"
//           (all other indices held to a fixed value) are consecutive in memory, where
//           consecutive values of "j" (fixing all other indices) are separated by
//           Nxx_plus_2NGHOSTS0 elements in memory. Similarly, consecutive values of
//           "k" are separated by Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1 in memory, etc.
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )
#define IDX4ptS(g,idx) ( (idx) + (Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2) * (g) )
#define IDX3S(i,j,k) ( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) ) ) )
#define LOOP_REGION(i0min,i0max, i1min,i1max, i2min,i2max) \
  for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++)
//#define LOOP_ALL_GFS_GPS(ii) _Pragma("omp parallel for") \
  for(int (ii)=0;(ii)<Nxx_plus_2NGHOSTS_tot*NUM_EVOL_GFS;(ii)++)

// Step P3: Set UUGF and VVGF macros, as well as xx_to_Cart()
//#include "boundary_conditions/gridfunction_defines.h"

// Step P4: Set xx_to_Cart(const paramstruct *restrict params,
//                     REAL *restrict xx[3],
//                     const int i0,const int i1,const int i2,
//                     REAL xCart[3]),
//           which maps xx->Cartesian via
//    {xx[0][i0],xx[1][i1],xx[2][i2]}->{xCart[0],xCart[1],xCart[2]}
#include "xx_to_Cart.h"

// Step P5: Defines set_Nxx_dxx_invdx_params__and__xx(const int EigenCoord, const int Nxx[3],
//                                       paramstruct *restrict params, REAL *restrict xx[3]),
//          which sets params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for
//          the chosen Eigen-CoordSystem if EigenCoord==1, or
//          CoordSystem if EigenCoord==0.
#include "set_Nxx_dxx_invdx_params__and__xx.h"

// Step P6: Include basic functions needed to impose curvilinear
//          parity and boundary conditions.
//#include "boundary_conditions/CurviBC_include_Cfunctions.h"
#include "set_bcstruct.h"
#include "MoL_step_forward_in_time.h"

// Step P7: Implement the algorithm for upwinding.
//          *NOTE*: This upwinding is backwards from
//          usual upwinding algorithms, because the
//          upwinding control vector in BSSN (the shift)
//          acts like a *negative* velocity.
//#define UPWIND_ALG(UpwindVecU) UpwindVecU > 0.0 ? 1.0 : 0.0

// Step P8: Include function for enforcing detgammahat constraint.
#include "enforce_detgammahat_constraint.h"

// Step P9: Find the CFL-constrained timestep
#include "find_timestep.h"

// Step P10: Declare initial data input struct:
//           stores data from initial data solver,
//           so they can be put on the numerical grid.
typedef struct __ID_inputs {
    int interp_stencil_size;
    int numlines_in_file;
    REAL *r_arr,*sf_arr,*psi4_arr,*alpha_arr;
} ID_inputs;

// Part P11: Declare all functions for setting up ScalarField initial data.
/* Routines to interpolate the ScalarField solution and convert to ADM & T^{munu}: */
#include "../nrpytutorial/ScalarField/ScalarField_interp.h"
#include "ID_scalarfield_ADM_quantities.h"
#include "ID_scalarfield_spherical.h"
#include "ID_scalarfield_xx0xx1xx2_to_BSSN_xx0xx1xx2.h"
#include "ID_scalarfield.h"

/* Next perform the basis conversion and compute all needed BSSN quantities */
#include "ID_ADM_xx0xx1xx2_to_BSSN_xx0xx1xx2__ALL_BUT_LAMBDAs.h"
#include "ID_BSSN__ALL_BUT_LAMBDAs.h"
#include "ID_BSSN_lambdas.h"



float fr[30000];
float fsf[30000];
float fpsi4[30000];
float falpha[30000];
//#include "NRPy_function_prototypes.h"

// Step P12: Set the generic driver function for setting up BSSN initial data
void initial_data(griddata_struct *restrict griddata) {

//Assinging pointers to griddata to match old way
    const paramstruct *params;
    params = &griddata->params;
    const bc_struct *bcstruct;
    bcstruct = &griddata->bcstruct;
    const rfm_struct *rfmstruct;
    rfmstruct = &griddata->rfmstruct;
    REAL *restrict xx[3];
    xx[0] = griddata->xx[0];
    xx[1] = griddata->xx[1];
    xx[2] = griddata->xx[2];
    REAL *restrict auxevol_gfs;
    auxevol_gfs = griddata->gridfuncs.auxevol_gfs;
    REAL *restrict in_gfs;
    in_gfs = griddata->gridfuncs.y_n_gfs;
#include "set_Cparameters.h"    
//We will change this to getter functions to grab the initial data from Leos initial conditions
    // Step 1: Set up ScalarField initial data
    // Step 1.a: Read ScalarField initial data from data file
    // Open the data file:
    // [Deleted to hard code in initial conditions]
    // Count the number of lines in the data file:
    
    
    // Allocate space for all data arrays:
    int numlines_in_file = 30000;
    REAL *r_arr     = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
    REAL *sf_arr    = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
    REAL *psi4_arr  = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
    REAL *alpha_arr = (REAL *)malloc(sizeof(REAL)*numlines_in_file);

    // Read from the data file, filling in arrays
    // read_datafile__set_arrays() may be found in ScalarField/ScalarField_interp.h
    #include "../SFID.h"
    //REAL *r_arr = r_arrjs;
    //REAL *sf_arr = sf_arrjs;
    //REAL *psi4_arr = psi4_arrjs;
    //REAL *alpha_arr = alpha_arrjs;
    
    //fprintf(stderr,"R_arr %e");
    
    for(int ln =0; ln<numlines_in_file;ln++){
        r_arr[ln] = fr[ln];
        sf_arr[ln] = fsf[ln];
        psi4_arr[ln] = fpsi4[ln];
        alpha_arr[ln] = falpha[ln];
    }
    //fprintf(stderr,"INITIAL_DATA %e %e %e %e TEST\n", r_arr[29999], sf_arr[29999], psi4_arr[29999],alpha_arr[29999]);
    
    const int interp_stencil_size = 12;
    ID_inputs SF_in;
    SF_in.interp_stencil_size = interp_stencil_size;
    SF_in.numlines_in_file    = numlines_in_file;
    SF_in.r_arr               = r_arr;
    SF_in.sf_arr              = sf_arr;
    SF_in.psi4_arr            = psi4_arr;
    SF_in.alpha_arr           = alpha_arr;

    // Step 1.b: Interpolate data from data file to set BSSN gridfunctions
    ID_scalarfield(params,xx,SF_in, in_gfs);
    ID_BSSN__ALL_BUT_LAMBDAs(params,xx,SF_in, in_gfs);
    //apply_bcs_curvilinear(params, bcstruct, NUM_EVOL_GFS, evol_gf_parity, in_gfs);
    const REAL dtt = 0;
    MoL_step_forward_in_time(griddata, dtt); //MoL applies BCs + steps forward, make dt=0
    enforce_detgammahat_constraint(rfmstruct, params,                   in_gfs);
    ID_BSSN_lambdas(params, xx, in_gfs);
    //apply_bcs_curvilinear(params, bcstruct, NUM_EVOL_GFS, evol_gf_parity, in_gfs);
    MoL_step_forward_in_time(griddata, dtt);
    enforce_detgammahat_constraint(rfmstruct, params,                   in_gfs);

    free(r_arr);
    free(sf_arr);
    free(psi4_arr);
    free(alpha_arr);
}

// Step P11: Declare function for evaluating Hamiltonian constraint (diagnostic)
//#include "Hamiltonian_constraint.h" (PROTOTYPE)

// Step P12: Declare rhs_eval function, which evaluates BSSN RHSs
//#include "rhs_eval.h" (PROTOTYPE)

// Step P13: Declare Ricci_eval function, which evaluates Ricci tensor
//#include "Ricci_eval.h" (PROTOTYPE)

//#include "NRPyCritCol_regridding.h"

REAL rho_max = 0.0;
//This is where the code diverges from the tutorial.
// (These used to be defined in main function, but to make them global they were moved outside main, and getter/setter 
// functions are used to modify them)
// Step P15: Declare global pointers and variables to be referenced in getter function.
//           This must be done in order to access variables created in the initialize
//           function in the stepfoward function.
int dimensions[4] = {28, 8, 2, 18}; //#Filler values that are updated later on

    
int arrNGHOSTS[4];
paramstruct params_p;
rfm_struct rfmstruct_p;
bc_struct bcstruct_p;
REAL N_final_p, output_every_N_p, dt_p;
//Contains scalar & metric fields, but also psi_4, so dont need the psi4 ones.
// Make resolution + cfl factor global variables. Generally make most variables global. 
REAL *y_n_gfs_p, *auxevol_gfs_p, *k_odd_gfs_p, *k_even_gfs_p, *y_nplus1_running_total_gfs_p, *xx_p[3];

// initialize() function:
// Step 0: Set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Set up initial data to an exact solutiondef
// Step 2: Initialize global pointers and variables

// Main function replaced by initialize and getter functions. Emscripten connects to Javascript.
int EMSCRIPTEN_KEEPALIVE initialize(REAL CFL_FACTOR){
    
    //fprintf(stderr,"Running initialize!!\n");
    griddata_struct griddata;
    paramstruct *params;
    params = &griddata.params;
//This is in prototype, but I like the old way more.
//Just need to change all params to griddata.params
#include "set_Cparameters_to_default.h"

    // Step 0a: Set up numerical grid structure, first in space...
    int n1 = dimensions[0];
    int n2 = dimensions[1];
    int n3 = dimensions[2];
    const int Nxx[3] = {n1,n2,n3};

    // Step 0b: Set free parameters, overwriting Cparameters defaults
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"

    // Step 0c: Uniform coordinate grids are stored to *xx[3]
    //REAL *restrict xx[3];
    //xx[0] = &griddata->xx[0];
    //xx[1] = &griddata->xx[1];
    //xx[2] = &griddata->xx[2];
    // Step 0c.i: Set bcstruct
    bc_struct *bcstruct;
    bcstruct = &griddata.bcstruct;
    {
        int EigenCoord = 1;
        // Step 0c.ii: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
        //             params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
        //             chosen Eigen-CoordSystem.
        
        set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, params, griddata.xx);
        
        // Step 0c.iii: Set Nxx_plus_2NGHOSTS_tot
//#include "set_Cparameters-nopointer.h"
        #include "set_Cparameters.h"
        const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;
        // Step 0d: Find ghostzone mappings; set up bcstruct
#include "set_up__bc_gz_map_and_parity_condns.h"
#include "driver_bcstruct.h"
        // Step 0d.i: Free allocated space for xx[][] array
        for(int i=0;i<3;i++) free(xx[i]);
    }
    
    // Step 0e: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    //          params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    //          chosen (non-Eigen) CoordSystem.
    int EigenCoord = 0;
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, params, xx);

    // Step 0f: Set all C parameters "blah" for params.blah, including
    //          Nxx_plus_2NGHOSTS0 = params.Nxx_plus_2NGHOSTS0, etc.
    #include "set_Cparameters-nopointer.h"
    const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

    // Step 0g: Time coordinate parameters
    const REAL t_final = 16; // Run until the user says stop, but let them be able to pause and continue, with reset button.

    // Step 0h: Set timestep based on smallest proper distance between gridpoints and CFL factor
    REAL dt = find_timestep(&params, xx, CFL_FACTOR);
    
    N_final_p = (int)(t_final / dt + 0.5); // The number of points in time.
                                             // Add 0.5 to account for C rounding down
                                             // typecasts to integers.
    //The Collision Code:
    //REAL out_approx_every_t = 0.2;
    //output_every_N_p = (int)(out_approx_every_t*((REAL)N_final_p)/t_final);
    //The Collapse code is below:
    //Every Nth step writes it to a file. For our version, instead of writing to a file just 
    //            1. Stores data 2. animates
    // We probably wont need to store data.
    REAL output_every_N = 20;
    
    

    // Step 0i: Error out if the number of auxiliary gridfunctions outnumber evolved gridfunctions.
    //              This is a limitation of the RK method. You are always welcome to declare & allocate
    //              additional gridfunctions by hand.
    
    if(NUM_AUX_GFS > NUM_EVOL_GFS) {
        fprintf(stderr,"Error: NUM_AUX_GFS > NUM_EVOL_GFS. Either reduce the number of auxiliary gridfunctions,\n");
        fprintf(stderr,"       or allocate (malloc) by hand storage for *diagnostic_output_gfs. \n");
        exit(1);
    }

    // Step 0j: Allocate memory for gridfunctions
    MoL_gridfunctions_struct *gridfuncs;
    gridfuncs = &griddata.gridfuncs;
#include "MoL_malloc_non_y_n_gfs.h"
#include "MoL_malloc_y_n_gfs.h"
    //REAL *restrict auxevol_gfs = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS_tot);
    //auxevol_gfs = &griddata.gridfuncs.auxevol_gfs;

    // Step 0k: Set up precomputed reference metric arrays
    // Step 0k.i: Allocate space for precomputed reference metric arrays.
    rfm_struct *rfmstruct;
    rfmstruct = &griddata.rfmstruct;
    rfm_precompute_rfmstruct_malloc(&params, &rfmstruct);

    // Step 1a: Set up initial data to an exact solution
    initial_data(&griddata);

    // Step 1b: Apply boundary conditions, as initial data
    //          are sometimes ill-defined in ghost zones.
    //          E.g., spherical initial data might not be
    //          properly defined at points where r=-1.
    //apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS,evol_gf_parity, y_n_gfs);
    MoL_step_forward_in_time(&griddata, 0);
    enforce_detgammahat_constraint(&griddata.rfmstruct, &griddata.params, griddata.gridfuncs.y_n_gfs);

    //Step 2: Assign pointers/Initialize global variables
    arrNGHOSTS[0] = NGHOSTS;
    arrNGHOSTS[1] = Nxx_plus_2NGHOSTS0;
    arrNGHOSTS[2] = Nxx_plus_2NGHOSTS1;
    arrNGHOSTS[3] = Nxx_plus_2NGHOSTS2;
    dt_p = dt; 
    //These appear to be all the pointers needed 
    params_p = params;
    rfmstruct_p = griddata.rfmstruct;
    bcstruct_p = griddata.bcstruct;
    y_n_gfs_p = griddata.gridfuncs.y_n_gfs;
    auxevol_gfs_p = griddata.gridfuncs.auxevol_gfs;
    k_odd_gfs_p = griddata.gridfuncs.k_odd_gfs;
    k_even_gfs_p = griddata.gridfuncs.k_even_gfs;
    y_nplus1_running_total_gfs_p = griddata.gridfuncs.y_nplus1_running_total_gfs;
    xx_p[0]=griddata.xx[0];
    xx_p[1]=griddata.xx[1];
    xx_p[2]=griddata.xx[2];
    //printf("%f",xx_p[0]);

    return 0;
}

// stepForward() function:
// Step 1: Define and initialize variables from initialize() function so they can be used in the RK-like Method
//         of Lines timestepping algorithm
// Step 2: Step forward one timestep (t -> t+dt) in time using chosen RK-like MoL timestepping algorithm
// Second half of main, does one single timescript that Javascript loops
void EMSCRIPTEN_KEEPALIVE stepForward(j){

    // Step 1: Redefine and initialize variables. In order to call each time-step one by one, we need to redefine
    //         some variables used in the MoL timestepping algorithm with the saved values from the initialization
    //         step
    int Nxx_plus_2NGHOSTS0 = arrNGHOSTS[1];
    int Nxx_plus_2NGHOSTS1 = arrNGHOSTS[2];
    int Nxx_plus_2NGHOSTS2 = arrNGHOSTS[3];
    int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;
    int N_final = N_final_p;
    int output_every_N = output_every_N_p;
    //REAL *restrict diagnostic_output_gfs = diagnostic_output_gfs_p;
    REAL dt = dt_p;
    paramstruct params = params_p;
    rfm_struct rfmstruct = rfmstruct_p;
    bc_struct bcstruct = bcstruct_p;
    REAL *y_n_gfs = y_n_gfs_p;
    REAL *restrict auxevol_gfs = auxevol_gfs_p;
    REAL *k_odd_gfs = k_odd_gfs_p;
    REAL *k_even_gfs = k_even_gfs_p;
    REAL *restrict y_nplus1_running_total_gfs = y_nplus1_running_total_gfs_p;
    REAL *xx[3];
    xx[0]=xx_p[0];
    xx[1]=xx_p[1];
    xx[2]=xx_p[2];
    //if(j){
    //    size_t len = sizeof(y_n_gfs_p)/sizeof(y_n_gfs_p[0]);
    //    fprintf(stderr, "The length of the array is: %zu\n", len);
    //}
#include "driver_bcstruct.h"
    // Step 2: Step forward one timestep (t -> t+dt) in time using
    //           chosen RK-like MoL timestepping algorithm
    // Step 3.a: Output 2D data file periodically, for visualization
  //  if(n%output_every_N == 0) {
        // Evaluate Hamiltonian constraint violation
   //     Hamiltonian_constraint(&rfmstruct, &params, y_n_gfs,auxevol_gfs, diagnostic_output_gfs);

        //char filename[100];
        //sprintf(filename,"out%d-%08d.txt",Nxx[0],n);
        //const int i1mid=Nxx_plus_2NGHOSTS1/2;
        //const int i2mid=Nxx_plus_2NGHOSTS2/2;
        //FILE *fp = fopen(filename, "w");
        //for( int i0=NGHOSTS;i0<Nxx_plus_2NGHOSTS0-NGHOSTS;i0++) {
        //    const int idx  = IDX3S(i0,i1mid,i2mid);
        //    const REAL xx0 = xx[0][i0];
        //    REAL xCart[3];
        //    xx_to_Cart(&params,xx,i0,i1mid,i2mid,xCart);
        //    const REAL rr = sqrt( xCart[0]*xCart[0] + xCart[1]*xCart[1] + xCart[2]*xCart[2] );
        //    fprintf(fp,"%e %e %e %e %e %e %e\n",xx0,rr,
        //            y_n_gfs[IDX4ptS(SFGF,idx)],y_n_gfs[IDX4ptS(SFMGF,idx)],
        //            y_n_gfs[IDX4ptS(ALPHAGF,idx)],y_n_gfs[IDX4ptS(CFGF,idx)],
        //            log10(fabs(diagnostic_output_gfs[IDX4ptS(HGF,idx)])));
        //}
        //fclose(fp);
    //}

//#include "MoLtimestepping/RK_MoL.h"
MoL_step_forward_in_time(&griddata, dt);
}



// Getter functions used to access the data in javascript after compliling.

// getNFinal(): returns final time-step value
REAL EMSCRIPTEN_KEEPALIVE getNFinal(){
    return N_final_p;
}

// setNFinal(): sets the final time-step value
void EMSCRIPTEN_KEEPALIVE setNFinal(int i){
    N_final_p = i;
}

// getdt(): gets the value for dt
REAL EMSCRIPTEN_KEEPALIVE getdt(){
    return dt_p;
}

// getNGHOSTS(): returns desired dimensions including ghosts shells
REAL EMSCRIPTEN_KEEPALIVE getNGHOSTS(int i){
    return arrNGHOSTS[i];
}

// getFxVal(): returns desired function value at specfic point in space, index i is determined from the
//             IDX3S and IDX4ptS arrays
REAL EMSCRIPTEN_KEEPALIVE getFxVal(int i){
    return y_n_gfs_p[i];
}

// getIDX3S(): returns IDX3S index for a given point
REAL EMSCRIPTEN_KEEPALIVE getIDX3S(int i0, int i1, int i2){
    int Nxx_plus_2NGHOSTS0 = arrNGHOSTS[1];
    int Nxx_plus_2NGHOSTS1 = arrNGHOSTS[2];
    int Nxx_plus_2NGHOSTS2 = arrNGHOSTS[3];
    return IDX3S(i0,i1,i2);
}

// getIDX4ptS(): returns IDX4ptS index to be used in getFxVal() using information from getIDX3S()
REAL EMSCRIPTEN_KEEPALIVE getIDX4ptS(int fx, REAL idx){
    int Nxx_plus_2NGHOSTS0 = arrNGHOSTS[1];
    int Nxx_plus_2NGHOSTS1 = arrNGHOSTS[2];
    int Nxx_plus_2NGHOSTS2 = arrNGHOSTS[3];
    return IDX4ptS(fx,idx);
}

// getCart(): returns the cartesian version of a given spherical coordinate
REAL EMSCRIPTEN_KEEPALIVE getCart(int i0, int i1, int i2, int index){
    REAL xCart[3];
    xx_to_Cart(&params_p,xx_p,i0,i1,i2,xCart);
    return xCart[index];
}

// setDim(): setter function allowing the user to change the resolution of the simulation
void EMSCRIPTEN_KEEPALIVE setDim(int dim1, int dim2, int dim3){
    dimensions[0] = dim1;
    dimensions[1] = dim2;
    dimensions[2] = dim3;
    dimensions[3]= dim1/4;
}

// getDim(): getter function for the dimensions/resolution of the simulation
int EMSCRIPTEN_KEEPALIVE getDim(int i){
    return dimensions[i];
}

// setInitials(): setter functions for the initial conditions
void EMSCRIPTEN_KEEPALIVE setInitR(float i, int j){
    fr[j] = i;
}
void EMSCRIPTEN_KEEPALIVE setInitSf(float i, int j){
    fsf[j] = i;
}
void EMSCRIPTEN_KEEPALIVE setInitPsi4(float i, int j){
    fpsi4[j] = i;
}
void EMSCRIPTEN_KEEPALIVE setInitAlpha(float i, int j){
    falpha[j] = i;
}
// A very crude attempt that hopefully returns the scalar field values
//void EMSCRIPTEN_KEEPALIVE getSf(int i){
//    return sf[i];
//} (It did not return the scalar field)

// runsim(): function that runs the initialize function with a specific resolution and CFL factor
void EMSCRIPTEN_KEEPALIVE runsim(){
    CFL_FACTOR = 1.0;
    
    fprintf(stderr,"Running Sim3!\n");
    
    initialize(CFL_FACTOR);
    
    fprintf(stderr,"Finished Sim!\n");
}

Overwriting SommerBSSN_Scalar_Collapse//ScalarFieldCollapse_Playground.c


In [138]:
import cmdline_helper as cmd 
main_c_file = os.path.join(Ccodesdir,"ScalarFieldCollapse_Playground.c")
main_output_file = os.path.join(outdir,"ScalarFieldCollapse.html")
print("Attempting to compile\n", main_c_file, "\nto\n", main_output_file,"\n")
cmd.C_compile(main_c_file, main_output_file, compile_mode="emscripten")

print("\nFiles in output directory are:\n", os.listdir(outdir))

Attempting to compile
 SommerBSSN_Scalar_Collapse/ScalarFieldCollapse_Playground.c 
to
 SommerBSSN_Scalar_Collapse/output/ScalarFieldCollapse.html 

Compiling executable...
(EXEC): Executing `emcc -std=gnu99 -s -O3 -march=native -funroll-loops -s ALLOW_MEMORY_GROWTH=1 SommerBSSN_Scalar_Collapse/ScalarFieldCollapse_Playground.c -o SommerBSSN_Scalar_Collapse/output/ScalarFieldCollapse.html -lm `...
In file included from SommerBSSN_Scalar_Collapse/ScalarFieldCollapse_Playground.c:269:
SommerBSSN_Scalar_Collapse/set_up__bc_gz_map_and_parity_condns.h:3:175: error: function definition is not allowed here
void set_up__bc_gz_map_and_parity_condns(const paramstruct *restrict params, REAL *restrict xx[3], gz_map *restrict bc_gz_map,parity_condition *restrict bc_parity_conditions){
                                                                                                                                                                              ^
In file included from SommerBSSN_Scalar_Co

SystemExit: 1

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb

In [ ]:
os.path.isfile(main_output_file)

In [ ]:
main_output_file